In [1]:
# Install necessary packages
!pip install flask flask-ngrok diffusers omegaconf transformers accelerate scipy safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144555 sha256=b22e64d25b3c28235fa48e178428b00a2dbaa06ed0b3f7a2c2fdfa0d06660c02
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Download ngrok and set up the authtoken
!wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq -n ngrok-stable-linux-amd64.zip
!./ngrok authtoken 2YnL6GaMcopUA2ae5wT7yzhARam_48jVuwcaZWdnb1DmTXMM1

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
from flask import Flask, request, jsonify, render_template
from diffusers import StableDiffusionPipeline
import torch
import gc
from flask_ngrok import run_with_ngrok
import base64
from io import BytesIO


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [5]:
device = "cuda"
m15= "/content/drive/MyDrive/Models/SD_1.5.safetensors"
m2="/content/drive/MyDrive/Models/SD_2.safetensors"
model15 = StableDiffusionPipeline.from_single_file(m15).to(device)
model12 = StableDiffusionPipeline.from_single_file(m2).to(device)

torch.cuda.empty_cache()
gc.collect()

config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

...

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:135: FutureWarning: Accessing config attribute `requires_safety_checker` directly via 'StableDiffusionPipeline' object attribute is deprecated. Please access 'requires_safety_checker' over 'StableDiffusionPipeline's config object instead, e.g. 'scheduler.config.requires_safe

866

In [6]:
# Create directories
!mkdir -p /content/templates
!mkdir -p /content/static

# Copy files from Google Drive to the Colab environment
!cp "/content/drive/My Drive/Models/index.html" /content/templates/index.html
!cp "/content/drive/My Drive/Models/styles.css" /content/static/styles.css
!cp "/content/drive/My Drive/Models/script.js" /content/static/script.js


In [7]:
# Initialize Flask app
app = Flask(__name__)
app.config['THREADS_PER_PAGE'] = 2
run_with_ngrok(app)  # Start ngrok when the app is run

In [8]:
@app.route('/generate', methods=['POST'])
def generate():
    data = request.get_json()
    prompt = data['prompt']
    model_name = data['model']

    if model_name == "1":
      with torch.no_grad():
          generated_images = model15(prompt)
          generated_image = generated_images.images[0]

    elif model_name == "2":
      with torch.no_grad():
          generated_images = model12(prompt)
          generated_image = generated_images.images[0]

    # Convert to base64
    buffered = BytesIO()
    generated_image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode()

    # Clear VRAM
    torch.cuda.empty_cache()
    gc.collect()

    return jsonify({'image_base64': img_str})

In [9]:
@app.route('/')
def index():
    return render_template('index.html')

In [ ]:
@app.route('/')
def index():
    return '''
    <!DOCTYPE html>
<html>
<head>
    <title>Image Generator</title>
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" rel="stylesheet">
    <script>
        async function generateImage() {
            let prompt = document.getElementById('prompt').value;
            let model = document.getElementById('model-select').value;

            let response = await fetch('/generate', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                },
                body: JSON.stringify({ prompt: prompt, model: model}),
            });
            let data = await response.json();
            let imageSrc = 'data:image/png;base64,' + data.image_base64;
            document.getElementById('image').src = imageSrc;

            let downloadLink = document.getElementById('download-link');
            downloadLink.href = imageSrc;
            downloadLink.download = 'generated-image.png';
            downloadLink.style.display = 'block';
        }


        function adjustTextArea() {
            const textArea = document.getElementById('prompt');
            const remainingChars = 200 - textArea.value.length;

            if (remainingChars >= 0) {
                textArea.setCustomValidity('');
            } else {
                textArea.setCustomValidity(`Exceeded character limit by ${Math.abs(remainingChars)} characters`);
            }

            textArea.reportValidity();
        }
    </script>
</head>
<body>
<div class="container mt-5">
        <h1 class="text-center mb-4">Generate Image</h1>
        <textarea id="prompt" class="form-control mb-3" rows="4" placeholder="Enter text"></textarea>
        <select id="model-select" class="form-control mb-3">
            <option value="1">SD 1.5</option>
            <option value="2">SD 2</option>
            <option value="3" disabled>SD 2.1 (Not Enough Computational Power)</option>
            <option value="4" disabled>SD 2.1 base (Not Enough Computational Power)</option>
            <option value="5" disabled>SD XL 0.9 base (Not Enough Computational Power)</option>
            <option value="6" disabled>SD XL 1.0 base (Not Enough Computational Power)</option>
        </select>
        <button onclick="generateImage()" class="btn btn-primary btn-block">Send</button>
        <br><br>
        <img id="image" class="img-fluid" />
        <br><br>
        <a id="download-link" class="btn btn-success mt-3" style="display: none;">Download Image</a>
    </div>
</body>
</html>

    '''

In [ ]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://cad0-35-194-248-108.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [30/Nov/2023 06:53:31] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Nov/2023 06:53:31] "GET /static/script.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Nov/2023 06:53:31] "GET /static/styles.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Nov/2023 06:53:32] "GET /favicon.ico HTTP/1.1" 404 -


  0%|          | 0/50 [00:00<?, ?it/s]

...

INFO:werkzeug:127.0.0.1 - - [30/Nov/2023 06:56:14] "POST /generate HTTP/1.1" 200 -


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [30/Nov/2023 06:59:18] "POST /generate HTTP/1.1" 200 -


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [30/Nov/2023 07:00:48] "POST /generate HTTP/1.1" 200 -
